In [ ]:
import caiman as cm
from caiman.base.rois import register_multisession
from caiman.utils import visualization
from caiman.utils.utils import download_demo
from matplotlib import pyplot as plt
import numpy as np
from caiman.source_extraction.cnmf.cnmf import load_CNMF
from caiman.mmapping import save_memmap
from scipy.sparse import csc_matrix
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
import os
from scipy.io import savemat

#start parallel processing, should just make things faster

###STARTS CLUSTERS 
        #%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)
#%% restart cluster to clean up memory
cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

2023-04-19 10:21:26.974841: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [33]:
#paths to the h5 files
fileList = ['/mnt/Brenna_Feb2023/TdTG7_19/20230327/gui/caimanOuputs_2023_4_2_16_1/ConcatenatedMovie_MC_reg_caimanEval_Python.hdf5', 
'/mnt/Brenna_Feb2023/TdTG7_19/20230327/track_beforegain/caimanOuputs_2023_4_2_18_9/ConcatenatedMovie_MC_reg_caimanEval_Python.hdf5']

sessionList = []
for file in fileList:
    currentEstimate = load_CNMF(file)
    sessionList.append(currentEstimate)

#paths to the movies
imageFileList = ['/mnt/Brenna_Feb2023/TdTG7_19/20230327/gui/ConcatenatedMovie_MC_reg.tif', 
'/mnt/Brenna_Feb2023/TdTG7_19/20230327/track_beforegain/ConcatenatedMovie_MC_reg.tif']

for cF in range(len(imageFileList)):
    file = imageFileList[cF]
    dataDir = os.path.dirname(file)
    fileName = os.path.basename(file)[0:-4]
    currentMemMap = save_memmap([file], base_name=fileName + 'mmapFile', order='C')

    #make the correlation image
    Yr, dims, T = cm.load_memmap(currentMemMap)
    images = np.reshape(Yr.T, [T] + list(dims), order='F')
    Cn = cm.local_correlations(images.transpose(1,2,0)) 
    #save it into the caiman object
    currentCNM = sessionList[cF]
    currentCNM.Cn = Cn
    #resave h5 with the correlations image
    currentCNM.save(os.path.join(dataDir, fileName + '_caimanEval_Python_testingRegisterFile.hdf5'))
    #should remove the memmapped file once the correlations imagee is made
    os.remove(os.path.join(dataDir ,currentMemMap))

In [42]:
#After your files are memory mapped and the new h5 file with the correlations field is saved you can put the paths to those h5 files in session list and run this cell. 
sessionList = []
CI = []
spatialComps = []
for cS in sessionList:
    currentSession = load_CNMF(cS)
    CI.append(currentSession.Cn)
    spatialComps.append(currentSession.estimates.A)


A = [csc_matrix(A1/A1.sum(0)) for A1 in spatialComps]

dims = [ci.shape for ci in CI]
dims_comb = tuple(np.array(dims).max(axis=0))

#this is going to do a lot of reshaping and padding because all the videos are not the same size a lot of the time
masks = [np.reshape(A_.toarray(), dims_ + (-1,), order='F').transpose(2, 0, 1)
             for A_, dims_ in zip(A, dims)]
masks = [np.pad(m, list(zip([0]*3,[0, *(np.array(dims_comb) - m.shape[1:])])),
                'constant', constant_values=0) for m in masks]
CI = [np.pad(ci, list(zip([0]*2, (np.array(dims_comb) - ci.shape))),
             'constant', constant_values=0) for ci in CI]

A = [csc_matrix(m.transpose(1,2,0).reshape((-1, m.shape[0]), order='F')) for m in masks]

#max threshold parameter before binarization (not totally sure what this means but seems like it is important)
max_thr = 0.1
#the actual line that does all of the registrations stuff
spatial_union, assignments, matchings = register_multisession(A, dims_comb, CI, max_thr=max_thr)
#This link could be helpful for understanding the outputs: https://caiman.readthedocs.io/en/master/core_functions.html#caiman.base.rois.register_multisession

matInfo = {}
matInfo['spatial_union'] = spatial_union
matInfo['assignments'] = assignments
matInfo['matchings'] = matchings

savemat(os.path.join(dataDir, fileName + 'registration.mat'), matInfo)


/home/howelab/anaconda3/envs/caiman/lib/python3.10/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
#we can then also filter those outputs and say well we only want ROIs that are active in 5 out of 6 sessions (or in this case 2)

# Filter components by number of sessions the component could be found

n_reg = 2  # minimal number of sessions that each component has to be registered in

# Use number of non-NaNs in each row to filter out components that were not registered in enough sessions
assignments_filtered = np.array(np.nan_to_num(assignments[np.sum(~np.isnan(assignments), axis=1) >= n_reg]), dtype=int)

# Use filtered indices to select the corresponding spatial components
spatial_filtered = A[0][:, assignments_filtered[:, 0]]

# Plot spatial components of the selected components on the template of the last session
visualization.plot_contours(spatial_filtered, CI[-1])

In [44]:
#useful functions for plotting an image and the ROIs you want over it
plot = nb_plot_contour(CI[0], spatial_filtered, dims_comb[0], dims_comb[1])

In [41]:
#Load individual caiman sessions (the h5)

# session1= load_CNMF('/mnt/Brenna_Feb2023/TdTG7_19/20230327/gui/ConcatenatedMovie_MC_reg_caimanEval_Python_testingRegisterFile.hdf5')
# session2 = load_CNMF('/mnt/Brenna_Feb2023/TdTG7_19/20230327/track_beforegain/ConcatenatedMovie_MC_reg_caimanEval_Python_testingRegisterFile.hdf5')